##  Игра Dota 2

[Dota 2](https://ru.wikipedia.org/wiki/Dota_2) — многопользовательская компьютерная игра жанра [MOBA](https://ru.wikipedia.org/wiki/MOBA). Игроки играют между собой матчи. В каждом матче участвует две команды, 5 человек в каждой. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника (трон).

Существуют [разные режимы игры](http://dota2.gamepedia.com/Game_modes/ru), мы будем рассматривать режим [Captain's Mode](http://dota2.gamepedia.com/Game_modes/ru#Captain.27s_Mode), в формате которого происходит большая часть киберспортивных мероприятий по Dota 2.

### Как проходит матч

#### 1. Игроки выбирают героев

Всего в игре чуть более 100 различных героев (персонажей). В начале игры, команды в определенном порядке выбирают героев себе и запрещают выбирать определенных героев противнику (баны). Каждый игрок будет управлять одним героем, в рамках одного матча не может быть несколько одинаковых героев.  Герои различаются между собой своими характеристиками и способностями. От комбинации выбранных героев во многом зависит успех команды.

![](http://imgur.com/XFr4HYE.jpg)

#### 2. Основная часть

Игроки могут получать золото и опыт за убийство чужих героев или прочих юнитов. Накопленный опыт влияет на уровень героя, который в свою очередь позволяет улучшать способности. За накопленное золото игроки покупают предметы, которые улучшают характеристики героев или дают им новые способности.

После смерти герой отправляется в "таверну" и возрождается только по прошествии некоторого времени, таким образом команда на некоторое время теряет игрока, однако игрок может досрочно выкупить героя из таверны за определенную сумму золота.

В течение игры команды развивают своих героев, обороняют свою часть поля и нападают на вражескую.

![](http://imgur.com/5b0SlQb.jpg)

#### 3. Конец игры

Игра заканчивается, когда одна из команд разрушет определенное число "башен" противника и уничтожает трон.

![](http://imgur.com/Du79Kzf.jpg)

## Задача: предсказание победы по данным о первых 5 минутах игры

По первым 5 минутам игры предсказать, какая из команд победит: Radiant или Dire?

## Набор данных

Набор данных с матчами записан в файле `matches.jsonlines.bz2`.
В каталоге `dictionaries` приведены расшифровки идентификаторов, которые присутствуют в записях матчей.

#### Чтение информации о матчах

Информация о матчах записана в сжатом текстовом файле `matches.jsonlines.bz2`, каждая строчка которого содержит объект в формате [JSON](https://ru.wikipedia.org/wiki/JSON). Запись в формате JSON преобразуется в python-объект при помощи стандартного модуля `json`. Пример чтения матчей:

In [5]:
import json
import bz2

with bz2.BZ2File('./matches.jsonlines.bz2') as matches_file:
    for line in matches_file:
        match = json.loads(line)
        
        # Обработка матча
        break

#### Описание полей в записи матча

```python
{
    "match_id": 247,            # идентификатор матча
    "start_time": 1430514316,   # дата/время начала матча, unixtime
    "lobby_type": 0,            # тип комнаты, в которой собираются игроки 
                                #   (расшифровка в dictionaries/lobbies.csv)
 
    # стадия выбора героев
    "picks_bans": [
        {
            "order": 0,       # порядковый номер действия
            "is_pick": false, # true если команда выбирает героя, false — если банит
            "team": 1,        # команда, совершающая действие (0 — Radiant, 1 — Dire)
            "hero_id": 95     # герой, связанный с действием 
                              #    (расшифровка в dictionaries/heroes.csv)
        }, 
        ...
    ],

    # информация про каждого игрока, список ровно из 10 элементов
    # игроки с индексами от 0 до 4 — из команды Radiant, от 5 до 9 — Dire
    "players": [ 
        { 
        
            # герой игрока (расшифровка в dictionaries/heroes.csv)
            "hero_id": 67,  

            # временные ряды (отсчеты указаны в поле "times")
            "xp_t": [0, 13, 115, 177, 335, ...],   # опыт
            "gold_t": [0, 99, 243, 343, 499, ...], # золото + стоимость всех купленных вещей (net worth)
            "lh_t": [0, 0, 2, 2, 2, ...],          # количество убитых юнитов (не героев) противника

            # список событий: улучшение способностей героя
            "ability_upgrades": [
                {
                    "time": 51,      # игровое время
                    "level": 1,      # уровень игрока, на котором произошло улучшение
                    "ability": 5334  # способность, которая была улучшена 
                                     # (расшифровка в dictionaries/abilities.csv)
                }, 
                ...
            ],

            # список событий: убийства
            "kills_log": [
                {
                    "time": 831,    # игровое время
                    "player": 7,    # индекс игрока, чей герой был убит 
                                    #   (не заполнено, если был убит не герой)
                    "unit": "npc_dota_hero_viper" # тип убитого юнита
                }, 
                ...
            ],

            # список событий: покупка предметов
            "purchase_log": [
                {
                    "time": -73,     # игровое время
                                     #   точка отсчета игрового времени (ноль) начинается через
                                     #   несколько минут после фактического начала матча, поэтому
                                     #   время некоторых событий может быть отрицательным
                    "item_id": 44    # купленный предмер (расшифровка в dictionaries/items.csv)
                }, 
                ...
            ]

            # список событий: выкуп героя из таверны
            "buyback_log": [
                {"time": 2507},
                ...
            ],

            # список событий: установка героем "наблюдателей", позволяющих команде 
            # следить за чатью игрового поля на некотором расстоянии от точки установки
            "obs_log": [
                {
                    "time": 1711,    # игровое время установки
                    "xy": [111, 130] # координаты игрового поля
                }, 
                ...
            ],
            "sen_log": [], # аналогично полю obs_log, другой тип "наблюдателя"

        },
        ...
    ],
    
    # отсчеты игрового времени, в которые вычисляются значения временных рядов
    "times": [0, 60, 120, 180, ...],

    # ключевые события игры
    "objectives": [
        {
            "time": 198,           # время события
            "type": "firstblood",  # тип события
            "player1": 6,          # параметры события, могут содержать
            "player2": 1           #   индексы игроков (player), 
                                   #   номер команды (team, 0 — Radiant, 1 — Dire)
        }, 
        {
            "time": 765, 
            "type": "tower_kill", 
            "player": 7, 
            "team": 1
        }, 
        ...
    ]
    
    # итог матча (отсутствует в тестовых матчах)
    "finish": {
        "duration": 2980,             # длительность в секундах
        "radiant_win": false,         # true, если победила команда Radiant
        "tower_status_radiant": 0,    # состояние башен у команд к концу игры
        "tower_status_dire": 1972,    #   (см. описание битовой маски)
        "barracks_status_dire": 63,   # состояние бараков у команд к концу игры
        "barracks_status_radiant": 0  #   (см. описание битовой маски)
    }
}
```

#### Описание полей состояния башен и бараков

Состояние башен к концу игры задается целым числом, закодировано в битах:

```
┌─┬─┬─┬─┬─────────────────────── Not used.
│ │ │ │ │ ┌───────────────────── Ancient Bottom
│ │ │ │ │ │ ┌─────────────────── Ancient Top
│ │ │ │ │ │ │ ┌───────────────── Bottom Tier 3
│ │ │ │ │ │ │ │ ┌─────────────── Bottom Tier 2
│ │ │ │ │ │ │ │ │ ┌───────────── Bottom Tier 1
│ │ │ │ │ │ │ │ │ │ ┌─────────── Middle Tier 3
│ │ │ │ │ │ │ │ │ │ │ ┌───────── Middle Tier 2
│ │ │ │ │ │ │ │ │ │ │ │ ┌─────── Middle Tier 1
│ │ │ │ │ │ │ │ │ │ │ │ │ ┌───── Top Tier 3
│ │ │ │ │ │ │ │ │ │ │ │ │ │ ┌─── Top Tier 2
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ┌─ Top Tier 1
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
```

Состояние бараков к концу игры закодировано в битах целого числа:

```
┌─┬───────────── Not used.
│ │ ┌─────────── Bottom Ranged
│ │ │ ┌───────── Bottom Melee
│ │ │ │ ┌─────── Middle Ranged
│ │ │ │ │ ┌───── Middle Melee
│ │ │ │ │ │ ┌─── Top Ranged
│ │ │ │ │ │ │ ┌─ Top Melee
│ │ │ │ │ │ │ │
0 0 0 0 0 0 0 0
```

## Извлечение признаков

Скрипт extract_features.py производит извлечение признаков из известной информации о матче за первые 5 игровых минут, составляет из них таблицу. 

Признаки, представленные в таблице `features.csv`, по мнению экспертов в предметной области являются наиболее важными для решения задачи предсказания победы команды. Тем не менее, не обязательно использовать эти признаки в исходном виде для применения методов машинного обучения — можно сделать новые признаки из имеющихся. Более того, признаки в файле `features.csv` содержат не всю информацию, известную про матч за первые 5 игровых минут. Можно использовать скрипт `extract_features.py` как пример и добавлять свои признаки для улучшения качества предсказания.

#### Пример чтения файла с признаками

In [1]:
import pandas
features = pandas.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


#### Описание признаков в таблице

- `match_id`: идентификатор матча в наборе данных
- `start_time`: время начала матча (unixtime)
- `lobby_type`: тип комнаты, в которой собираются игроки (расшифровка в `dictionaries/lobbies.csv`)
- Наборы признаков для каждого игрока (игроки команды Radiant — префикс `rN`, Dire — `dN`):
    - `r1_hero`: герой игрока (расшифровка в dictionaries/heroes.csv)
    - `r1_level`: максимальный достигнутый уровень героя (за первые 5 игровых минут)
    - `r1_xp`: максимальный полученный опыт
    - `r1_gold`: достигнутая ценность героя
    - `r1_lh`: число убитых юнитов
    - `r1_kills`: число убитых игроков
    - `r1_deaths`: число смертей героя
    - `r1_items`: число купленных предметов
- Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
- Признаки для каждой команды (префиксы `radiant_` и `dire_`)
    - `radiant_bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_courier_time`: время приобретения предмета "courier" 
    - `radiant_flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_tpscroll_count`: число предметов "tpscroll" за первые 5 минут
    - `radiant_boots_count`: число предметов "boots"
    - `radiant_ward_observer_count`: число предметов "ward_observer"
    - `radiant_ward_sentry_count`: число предметов "ward_sentry"
    - `radiant_first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

## Метрика качества

В качестве метрики качества мы будем использовать площадь под ROC-кривой (AUC-ROC). Обратим внимание, что AUC-ROC — это метрика качества для алгоритма, выдающего оценки принадлежности первому классу. Оба алгоритма, которые будут использоваться в проекте — градиентный бустинг, и логистическая регрессия — умеют выдавать такие оценки. Для этого нужно получать предсказания с помощью функции predict_proba. Она возвращает два столбца: первый содержит оценки принадлежности нулевому классу, второй — первому классу. Нам нужны значения из второго столбца:
```python
pred = clf.predict_proba(X_test)[:, 1]
```

### Подход 1: градиентный бустинг "в лоб"
Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.

   
##### Измерение времени работы кода
```python
import time
import datetime

start_time = datetime.datetime.now()

time.sleep(3) # вместо этой строчки разместить замеряемый код

print 'Time elapsed:', datetime.datetime.now() - start_time
```

# Подход 1

1. Считывание признаков из features.csv и удаление признаков с итогами матча.

In [243]:
import pandas as pd
features = pd.read_csv('features.csv', index_col='match_id')
y = features['radiant_win']
features = features.drop(features.columns[[102,103,104,105,106,107]], axis=1)
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


2. Проверка выборки на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Записать названия признаков, имеющих пропуски, и попробовать для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [218]:
counts = features.count()
for i in range (0, len(counts) - 1):
    if counts[i] != len(features):
        print("Column: {0}. Count of missing values: {1}".format(counts.index[i], len(features) - counts[i]))

Column: first_blood_time. Count of missing values: 19553
Column: first_blood_team. Count of missing values: 19553
Column: first_blood_player1. Count of missing values: 19553
Column: first_blood_player2. Count of missing values: 43987
Column: radiant_bottle_time. Count of missing values: 15691
Column: radiant_courier_time. Count of missing values: 692
Column: radiant_flying_courier_time. Count of missing values: 27479
Column: radiant_first_ward_time. Count of missing values: 1836
Column: dire_bottle_time. Count of missing values: 16143
Column: dire_courier_time. Count of missing values: 676
Column: dire_flying_courier_time. Count of missing values: 26098


Посмотрим на описание признаков:
Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
first_blood_time: игровое время первой крови
first_blood_team: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
first_blood_player1: игрок, причастный к событию
first_blood_player2: второй игрок, причастный к событию

Как видно из кода выше, в данных признаках есть пропуски, что свидетельствует о том, что за первые 5 минут данное событие не успело произойти.

radiant_bottle_time: время первого приобретения командой предмета "bottle" - за первые 5 минут данный предмет приобретен не был.
radiant_courier_time: время приобретения предмета "courier" - аналогично.

3. Заменить пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака.

In [244]:
features = features.fillna(0)
features.head(3)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0


4. Столбец с целевой переменной.

In [215]:
y.head(5)
#radiant_win

match_id
0    1
1    1
2    0
3    0
4    0
Name: radiant_win, dtype: int64

5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируем генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудем перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. 

In [67]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np

In [94]:
import time
import datetime
def cv_estimate(n_splits, trees):
    roc_auc = []
    cv = KFold(n_splits=n_splits, shuffle=True)
    cv_clf = GradientBoostingClassifier(n_estimators=trees)
    for train, test in cv.split(features, y):
        start_time = datetime.datetime.now()
        cv_clf.fit(features.iloc[train], y.iloc[train])
        pred = cv_clf.predict_proba(features.iloc[test])[:, 1]
        print('Time with ' + str(n) + "trees: " + str(datetime.datetime.now() - start_time))
        roc_auc.append(roc_auc_score(y.iloc[test], pred))
    res = np.array(roc_auc).mean()
    print ("Roc_auc: " + str(res))
    return res


In [95]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
trees =  [10, 20, 30]
for n in trees:
    print('n: ' + str(n))
    cv_estimate(n_splits=5, trees=n)
    

n: 10
Time with 10trees: 0:00:05.416000
Time with 10trees: 0:00:05.535000
Time with 10trees: 0:00:05.376000
Time with 10trees: 0:00:05.389000
Time with 10trees: 0:00:05.955000
Roc_auc: 0.6652101009493867
n: 20
Time with 20trees: 0:00:10.323000
Time with 20trees: 0:00:09.278000
Time with 20trees: 0:00:08.774000
Time with 20trees: 0:00:08.968000
Time with 20trees: 0:00:08.838000
Roc_auc: 0.6819614589349112
n: 30
Time with 30trees: 0:00:13.707000
Time with 30trees: 0:00:14.459000
Time with 30trees: 0:00:14.749000
Time with 30trees: 0:00:13.310000
Time with 30trees: 0:00:14.550000
Roc_auc: 0.6892048567318687


In [98]:
 print('n: ' + str(50))
cv_estimate(n_splits=5, trees=50)

n: 50
Time with 30trees: 0:00:22.618000
Time with 30trees: 0:00:21.974000
Time with 30trees: 0:00:21.961000
Time with 30trees: 0:00:22
Time with 30trees: 0:00:23.027000
Roc_auc: 0.697434776616755


0.697434776616755

Есть смысл использовать больше 30 деревьев в градиентном бустинге, так как качество растет, однако скорость существенно падает.
Для ускорения обучения можно использовать не все объекты, а лишь половину, или же изменить глубину дерева.

### Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

НО!  линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.
##### Код для формирования "мешка слов" по героям
```python
# N — количество различных героев в выборке
X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
```

1. Оценим качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберем при этом лучший параметр регуляризации (C). 

In [245]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(features)
features = scaler.transform(features)

In [177]:
def cv_estimate_log_reg(c, features):
    roc_auc = []
    cv = KFold(n_splits=5, shuffle=True)
    cv_lr = LogisticRegression(C=c)
    for train, test in cv.split(features, y):
        start_time = datetime.datetime.now()
        cv_lr.fit(features[train], y.iloc[train])
        pred = cv_lr.predict_proba(features[test])[:, 1]
        print('Time with C = ' + str(c) + ": " + str(datetime.datetime.now() - start_time))
        roc_auc.append(roc_auc_score(y.iloc[test], pred))
    res = np.array(roc_auc).mean()
    print ("Roc_auc: " + str(res))
    return res

In [178]:
from sklearn.linear_model import LogisticRegression
cv = KFold(n_splits=5, shuffle=True)
for n in np.logspace(-3, 0, num=5):
    print('C: ' + str(n))
    cv_estimate_log_reg(c=n, features = features)

C: 0.001
Time with C = 0.001: 0:00:02.137000
Time with C = 0.001: 0:00:02.086000
Time with C = 0.001: 0:00:02.001000
Time with C = 0.001: 0:00:01.656000
Time with C = 0.001: 0:00:02.033000
Roc_auc: 0.7162290223799299
C: 0.005623413251903491
Time with C = 0.005623413251903491: 0:00:02.657000
Time with C = 0.005623413251903491: 0:00:02.666000
Time with C = 0.005623413251903491: 0:00:02.789000
Time with C = 0.005623413251903491: 0:00:02.906000
Time with C = 0.005623413251903491: 0:00:02.519000
Roc_auc: 0.7163942341954745
C: 0.03162277660168379
Time with C = 0.03162277660168379: 0:00:03.008000
Time with C = 0.03162277660168379: 0:00:03.081000
Time with C = 0.03162277660168379: 0:00:02.879000
Time with C = 0.03162277660168379: 0:00:02.832000
Time with C = 0.03162277660168379: 0:00:02.903000
Roc_auc: 0.7163259726158111
C: 0.1778279410038923
Time with C = 0.1778279410038923: 0:00:02.959000
Time with C = 0.1778279410038923: 0:00:02.963000
Time with C = 0.1778279410038923: 0:00:02.916000
Time w

Качество, полученное линейной регрессией (~0.71), лучше, чем бустингом (max - ~0.69). Кроме того, по скорости линейная регрессия также выигрывает во много раз.


2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберем их из выборки, и проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. 

In [246]:
features = pd.read_csv('features.csv', index_col='match_id')
y = features['radiant_win']
features = features.drop(features.columns[[1,102,103,104,105,106,107]], axis=1)
features = features.fillna(0)
features.drop(['r1_hero', 'r2_hero', 
              'r3_hero',
              'r4_hero',
              'r5_hero',
              'd1_hero',
               'd2_hero',
              'd3_hero',
              'd4_hero',
              'd5_hero'], axis=1, inplace=True)
features = scaler.fit_transform(features)


In [180]:
from sklearn.linear_model import LogisticRegression
cv = KFold(n_splits=5, shuffle=True)
for n in np.logspace(-3, 0, num=5):
    print('C: ' + str(n))
    cv_estimate_log_reg(c=n, features = features)

C: 0.001
Time with C = 0.001: 0:00:01.887000
Time with C = 0.001: 0:00:01.853000
Time with C = 0.001: 0:00:01.867000
Time with C = 0.001: 0:00:01.897000
Time with C = 0.001: 0:00:01.856000
Roc_auc: 0.7163389138763048
C: 0.005623413251903491
Time with C = 0.005623413251903491: 0:00:02.384000
Time with C = 0.005623413251903491: 0:00:02.374000
Time with C = 0.005623413251903491: 0:00:02.339000
Time with C = 0.005623413251903491: 0:00:02.390000
Time with C = 0.005623413251903491: 0:00:02.281000
Roc_auc: 0.7165721226727395
C: 0.03162277660168379
Time with C = 0.03162277660168379: 0:00:02.604000
Time with C = 0.03162277660168379: 0:00:02.525000
Time with C = 0.03162277660168379: 0:00:02.533000
Time with C = 0.03162277660168379: 0:00:02.543000
Time with C = 0.03162277660168379: 0:00:02.446000
Roc_auc: 0.7163347151085289
C: 0.1778279410038923
Time with C = 0.1778279410038923: 0:00:02.288000
Time with C = 0.1778279410038923: 0:00:02.607000
Time with C = 0.1778279410038923: 0:00:02.103000
Time w

Удаление категориальных признаков практически не влияет на результат (упало на тысячную долю)

3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясним из данных, сколько различных идентификаторов героев существует в данной игре.

In [247]:
heroes = ['r1_hero', 'r2_hero', 
              'r3_hero',
              'r4_hero',
              'r5_hero',
              'd1_hero',
               'd2_hero',
              'd3_hero',
              'd4_hero',
              'd5_hero']
feat = pd.read_csv('features.csv', index_col='match_id')
feat = feat.drop(feat.columns[[1,102,103,104,105,106,107]], axis=1)
feat = feat.fillna(0)
heroes = feat[heroes]
unique_heroes = set()
for hero_feature_name in heroes:
    unique_heroes.update(feat[hero_feature_name].unique())
print(len(unique_heroes))
unique_heroes = list(unique_heroes)

108


4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire.

В данном случе N = 108

In [248]:
X_pick = np.zeros((feat.shape[0], 108))

for i, match_id in enumerate(feat.index):
    for p in xrange(5):
        
        r_hero_code = feat.loc[match_id, 'r%d_hero' % (p+1)]
        X_pick[i, unique_heroes.index(r_hero_code)] = 1
        
        d_hero_code = feat.loc[match_id, 'd%d_hero' % (p+1)]
        X_pick[i, unique_heroes.index(d_hero_code)] = -1

feat = np.hstack((feat, X_pick))
print (len(feat[0]))
feat_scaled = scaler.fit_transform(feat)

209


5. Проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. 

In [189]:
for n in np.logspace(-3, 0, num=5):
    print('C: ' + str(n))
    cv_estimate_log_reg(c=n, features = feat_scaled)

C: 0.001
Time with C = 0.001: 0:00:03.674000
Time with C = 0.001: 0:00:03.750000
Time with C = 0.001: 0:00:03.827000
Time with C = 0.001: 0:00:03.885000
Time with C = 0.001: 0:00:03.743000
Roc_auc: 0.7516882545644344
C: 0.005623413251903491
Time with C = 0.005623413251903491: 0:00:05.207000
Time with C = 0.005623413251903491: 0:00:05.283000
Time with C = 0.005623413251903491: 0:00:05.205000
Time with C = 0.005623413251903491: 0:00:05.312000
Time with C = 0.005623413251903491: 0:00:05.103000
Roc_auc: 0.7519159634002179
C: 0.03162277660168379
Time with C = 0.03162277660168379: 0:00:05.457000
Time with C = 0.03162277660168379: 0:00:05.978000
Time with C = 0.03162277660168379: 0:00:05.508000
Time with C = 0.03162277660168379: 0:00:05.899000
Time with C = 0.03162277660168379: 0:00:06.050000
Roc_auc: 0.7518419445805062
C: 0.1778279410038923
Time with C = 0.1778279410038923: 0:00:05.993000
Time with C = 0.1778279410038923: 0:00:06.180000
Time with C = 0.1778279410038923: 0:00:06.160000
Time w

Нетрудно заметить, что качество улучшилось (~0.75). Это объясняется тем, что добавленные признаки имеют важное значение для модели, что и улучшает качество

6. Построим предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедимся, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [280]:
feat_test = pd.read_csv('features_test.csv')
print len(feat_test)
feat_test.drop([
    'start_time',
    'lobby_type'], axis=1, inplace=True)
feat_test = feat_test.fillna(0)
feat_test.head(5)


17177


,match_id,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,6,93,4,1103,1089,8,0,1,9,102,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
1,7,20,2,556,570,1,0,0,9,6,...,2,-29.0,168.0,-54.0,0.0,3,2,2,1,16.0
2,10,112,2,751,808,1,0,0,13,26,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
3,13,27,3,708,903,1,1,1,11,91,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
4,16,39,4,1259,661,4,0,0,9,93,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [261]:
X_pick = np.zeros((feat_test.shape[0], 108))

for i, match_id in enumerate(feat_test.index):
    for p in xrange(5):
        
        r_hero_code = feat_test.loc[match_id, 'r%d_hero' % (p+1)]
        X_pick[i, unique_heroes.index(r_hero_code)] = 1
        
        d_hero_code = feat_test.loc[match_id, 'd%d_hero' % (p+1)]
        X_pick[i, unique_heroes.index(d_hero_code)] = -1
        
features_test = np.hstack((feat_test, X_pick))
feat_scaled_test = scaler.fit_transform(features_test)

In [262]:
clf = LogisticRegression(penalty='l2', C=0.005623413251903491)

In [263]:
clf.fit(feat_scaled, y)

LogisticRegression(C=0.0056234132519, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [264]:
probabilities = clf.predict_proba(feat_scaled_test)[:, 1]
print probabilities

[0.82551949 0.76215675 0.19985459 ... 0.23872483 0.63785534 0.43864583]


In [281]:
result = {'match_id': [], 'radiant_win': []}
df = feat_test.loc[:, 'match_id']
for i, match_id in enumerate(feat_test.index):
    result['match_id'].append(df[i])
    result['radiant_win'].append(probabilities[i])

In [283]:
res_df = pd.DataFrame(result)
print len(res_df)
print res_df.head(5)
res_df.set_index('match_id', inplace=True)
res_df.to_csv('final-results.csv')

17177
   match_id  radiant_win
0         6     0.825519
1         7     0.762157
2        10     0.199855
3        13     0.856204
4        16     0.244227


## Проверка финальной модели

После того как проведены все эксперименты и выбрана лучшая модель, можно проверить ее качество на тестовых матчах. Таблица признаков для тестовых матчей — `features_test.csv`.

Для всех матчей из тестового набора предскажем вероятность победы Radiant, запишем предсказания в CSV файл с колонками `match_id` (идентификатор матча) и `radiant_win` — предсказанная вероятность. Файл с предсказаниями должен выглядеть примерно следующим образом:

```
match_id,radiant_win
1,0.51997370502
4,0.51997370502
15,0.51997370502
...
```

### Что еще попробовать?

Разумеется, можно попробовать еще очень много разных идей, которые помогут получить еще более высокий результа. Вот лишь несколько возможных вариантов:
1. Про каждого из игроков есть достаточно много показателей: максимальный опыт, число смертей и т.д. (см. список выше). Можно попробовать просуммировать или усредних их, получив агрегированные показатели для всей команды.
2. В сырых данных (файл matches.jsonlines.bz2) содержится очень много информации, которую мы пока не использовали. Можно, например, составить "мешки слов" для покупок различных предметов (то есть кодировать информацию о том, сколько раз каждая команда покупала тот или иной предмет). Обратим внимание, что при этом можно получить слишком большое количество признаков, для которых может иметь смысл сделать понижение размерности с помощью метода главных компонент.
3. Можно сформировать признаки про изменения способностей героев в течение матча (ability_upgrades).
4. Можно попробовать метод k ближайших соседей, SVM, случайный лес и так далее.